In [1]:
import pandas as pd
from scipy import stats
# import custom libraries
from tools.glm import (add_regressors, array2dummies,
                       n_back_array, n_back_series,
                       glm, contrast)

df = pd.read_csv("../data/regs_and_rts.csv")

In [2]:
pes = pd.DataFrame()

for (sub, ses, scn), group in df.groupby(["sub", "real_ses", "scn"], as_index=False):
    X = group[["loc1", "loc2", "loc3", "loc4", "lag1_loc", "lag2_loc", "lag1_clr", "lag2_clr"]].astype(float)
    Y = group.norm_rt
    b = glm(Y, X.T)
    
    b[:4] * group[["loc1", "loc2", "loc3", "loc4"]]
    
    baseline = (b[:4] * group[["loc1", "loc2", "loc3", "loc4"]]).sum(axis=1)
    res = Y - baseline
    
    regs = X.columns.tolist()[4:]
    
    rs = []
    for i, reg in enumerate(regs):
        other = regs.copy()
        other.remove(reg)
        
        # Make use of b[i] corresponding to regs[i]
        b_res = list(b[4:])
        b_reg = b_res.pop(i)
        
        yhat_res = (b_res * group[other]).sum(axis=1)
        yhat_reg = (b_reg * group[reg])

        r, p = stats.pearsonr(res - yhat_res, yhat_reg)
        rs.append(r**2)
    
    result = pd.DataFrame([b], columns=["loc1", "loc2", "loc3", "loc4", "lag1_loc", "lag2_loc", "lag1_clr", "lag2_clr"])
    result = result.join(pd.DataFrame([rs], columns=[reg+"_r" for reg in regs]))
    result = result.assign(**{
        "sub": sub,
        "ses": ses,
        "scn": scn,        
    })
    
    pes = pes.append(result)